# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [14]:
import os
import sys
from utils import *

sys.path.append('..')
# Set your preferred name
USER = "Lisa"
# Set the data and output directories
DATA_ROOT = '/Users/wjt/Desktop/is_data'
OUTPUT_ROOT = '/Users/wjt/Desktop/is_data/v1'
TRAIT = 'Susceptibility-to-Infections'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [9]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [8]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [7]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

NameError: name 'clinical_data_file' is not defined

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [15]:
dataset = 'GEO'
trait_subdir = "Susceptibility-to-Infections"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE116725', 'GSE195434', 'GSE150133']

Repeat the below steps for all the accession numbers

In [17]:
cohort = accession_num = "GSE150133"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/wjt/Desktop/is_data/GEO/Susceptibility-to-Infections/GSE150133/GSE150133_family.soft.gz',
 '/Users/wjt/Desktop/is_data/GEO/Susceptibility-to-Infections/GSE150133/GSE150133_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [18]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Associations between IFI44L gene variants and rates of respiratory tract infections during early childhood"
!Series_summary	"Background: Genetic heterogeneity in the innate immune system may account for variable susceptibility to respiratory tract infections (RTIs) in children.  Objective: We aimed to assess the impact of polymorphisms rs273259 and rs1333969 in type I interferon related gene IFI44L on susceptibility to RTIs and acute otitis media in children.  Methods: In two prospective, population-based birth cohorts, the FinnBrain Birth Cohort Study and the STEPS Study, IFI44L genotypes for rs273259 and rs1333969 were determined in relation to the development of RTIs until one and two years of age, respectively, and adjusted incidence rate ratios (aIRR) or odds ratios (OR) were calculated. At age 3 months, whole blood transcriptional profiles were analyzed and nasal samples were tested for respiratory viruses in a subset of children.   Results: In respiratory virus-po

In [19]:
clinical_data.head()

,!Sample_geo_accession,GSM4524034,GSM4524035,GSM4524036,GSM4524037,GSM4524038,GSM4524039,GSM4524040,GSM4524041,GSM4524042,...,GSM4524095,GSM4524096,GSM4524097,GSM4524098,GSM4524099,GSM4524100,GSM4524101,GSM4524102,GSM4524103,GSM4524104
0,!Sample_characteristics_ch1,subject: 1,subject: 2,subject: 3,subject: 4,subject: 5,subject: 6,subject: 7,subject: 8,subject: 9,...,subject: 62,subject: 63,subject: 64,subject: 65,subject: 66,subject: 67,subject: 68,subject: 69,subject: 70,subject: 71
1,!Sample_characteristics_ch1,age in days: 79,age in days: 76,age in days: 77,age in days: 75,age in days: 81,age in days: 77,age in days: 73,age in days: 66,age in days: 83,...,age in days: 68,age in days: 67,age in days: 68,age in days: 75,age in days: 69,age in days: 62,age in days: 67,age in days: 66,age in days: 87,age in days: 67
2,!Sample_characteristics_ch1,gender: Male,gender: Female,gender: Male,gender: Male,gender: Male,gender: Female,gender: Male,gender: Male,gender: Male,...,gender: Female,gender: Male,gender: Female,gender: Male,gender: Female,gender: Female,gender: Male,gender: Female,gender: Male,gender: Male
3,!Sample_characteristics_ch1,race: White,race: White,race: White,race: White,race: White,race: White,race: White,race: White,race: White,...,race: White,race: White,race: White,race: White,race: White,race: White,race: White,race: White,race: White,race: White
4,!Sample_characteristics_ch1,respiratory virus detected: Yes,respiratory virus detected: Yes,respiratory virus detected: No,respiratory virus detected: No,respiratory virus detected: Yes,respiratory virus detected: Yes,respiratory virus detected: No,respiratory virus detected: No,respiratory virus detected: No,...,respiratory virus detected: No,respiratory virus detected: No,respiratory virus detected: No,respiratory virus detected: N/A,respiratory virus detected: No,respiratory virus detected: No,respiratory virus detected: Yes,respiratory virus detected: No,respiratory virus detected: No,respiratory virus detected: Yes


In [20]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['subject: 1',
  'subject: 2',
  'subject: 3',
  'subject: 4',
  'subject: 5',
  'subject: 6',
  'subject: 7',
  'subject: 8',
  'subject: 9',
  'subject: 10',
  'subject: 11',
  'subject: 12',
  'subject: 13',
  'subject: 14',
  'subject: 15',
  'subject: 16',
  'subject: 17',
  'subject: 18',
  'subject: 19',
  'subject: 20',
  'subject: 21',
  'subject: 22',
  'subject: 23',
  'subject: 24',
  'subject: 25',
  'subject: 26',
  'subject: 27',
  'subject: 28',
  'subject: 29',
  'subject: 30'],
 1: ['age in days: 79',
  'age in days: 76',
  'age in days: 77',
  'age in days: 75',
  'age in days: 81',
  'age in days: 73',
  'age in days: 66',
  'age in days: 83',
  'age in days: 88',
  'age in days: 86',
  'age in days: 82',
  'age in days: 78',
  'age in days: 80',
  'age in days: 107',
  'age in days: 84',
  'age in days: 72',
  'age in days: 89',
  'age in days: 69',
  'age in days: 64',
  'age in days: 97',
  'age in days: 70',
  'age in days: 100',
  'age in days: 85',
  'age 

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [21]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Susceptibility-to-Infections\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Susceptibility-to-Infections\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this t

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [22]:
is_gene_availabe = True
trait_row = 4
age_row = 1
gender_row = 2

trait_type = 'binary'

In [23]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [24]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'respiratory virus detected: Yes':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age in days: '):
        try:
            return int(age_string.split('age in days: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, 1)

In [25]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM4524034,GSM4524035,GSM4524036,GSM4524037,GSM4524038,GSM4524039,GSM4524040,GSM4524041,GSM4524042,GSM4524043,...,GSM4524095,GSM4524096,GSM4524097,GSM4524098,GSM4524099,GSM4524100,GSM4524101,GSM4524102,GSM4524103,GSM4524104
Susceptibility-to-Infections,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
Age,79,76,77,75,81,77,73,66,83,88,...,68,67,68,75,69,62,67,66,87,67
Gender,1,0,1,1,1,0,1,1,1,1,...,0,1,0,1,0,0,1,0,1,1


### Genetic data preprocessing and final filtering

In [26]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM4524034,GSM4524035,GSM4524036,GSM4524037,GSM4524038,GSM4524039,GSM4524040,GSM4524041,GSM4524042,GSM4524043,...,GSM4524095,GSM4524096,GSM4524097,GSM4524098,GSM4524099,GSM4524100,GSM4524101,GSM4524102,GSM4524103,GSM4524104
ID,,,,,,,,,,,,,,,,,,,,,
ILMN_1343291,13309.5195,9765.3945,8867.2051,13128.9766,9564.8379,9448.2520,11272.5820,10328.0078,9231.8906,10374.8604,...,9093.6113,11328.8477,9156.8330,14582.0225,8973.2021,12808.9590,9158.5527,10990.1729,8964.0518,9565.1475
ILMN_1343295,1561.3577,1255.9554,1131.8611,1360.7297,1160.3673,966.0376,706.0789,1161.2283,1087.5573,1035.1798,...,937.0262,1717.5266,999.5264,1172.7281,1205.9568,940.1333,1236.7032,1117.1794,908.4764,1299.9790
ILMN_1651199,-0.9195,-3.9747,-2.9242,-5.1920,-1.1818,-1.5697,-5.5167,-0.1118,-6.2358,2.3655,...,0.4762,2.2516,-4.4837,-0.1125,4.5636,5.8077,-3.4421,-2.4684,-1.5485,-3.3415
ILMN_1651209,-2.2325,2.9160,14.7379,2.5597,6.8982,-4.7455,1.3540,2.3672,-2.0607,9.1681,...,4.8679,7.7793,-4.8767,0.0761,5.3398,-1.0107,-0.3009,7.1235,8.4283,-5.3956
ILMN_1651210,0.9305,-1.0970,0.3967,-7.0590,-1.4322,3.4795,-5.5385,1.4125,-1.8344,-0.9941,...,0.0817,2.8319,7.6631,-1.7962,6.5084,12.0006,3.6063,7.5671,-0.6048,4.5391


In [27]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ILMN_1343291',
 'ILMN_1343295',
 'ILMN_1651199',
 'ILMN_1651209',
 'ILMN_1651210',
 'ILMN_1651221',
 'ILMN_1651228',
 'ILMN_1651229',
 'ILMN_1651230',
 'ILMN_1651232',
 'ILMN_1651235',
 'ILMN_1651236',
 'ILMN_1651237',
 'ILMN_1651238',
 'ILMN_1651249',
 'ILMN_1651253',
 'ILMN_1651254',
 'ILMN_1651259',
 'ILMN_1651260',
 'ILMN_1651262']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [28]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209', 'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262']\n"

If not required, jump directly to the gene normalization step

In [29]:
requires_gene_mapping = True

In [30]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['ILMN_1762337', 'ILMN_2055271', 'ILMN_1736007', 'ILMN_2383229', 'ILMN_1806310'], 'SPECIES': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'SOURCE': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'SEARCH_KEY': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_138933.1'], 'TRANSCRIPT': ['ILMN_183371', 'ILMN_175569', 'ILMN_18893', 'ILMN_18532', 'ILMN_7300'], 'ILMN_GENE': ['7A5', 'A1BG', 'A1BG', 'A1CF', 'A1CF'], 'SOURCE_REFERENCE_ID': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_014576.2'], 'REFSEQ_ID': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_014576.2'], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENTREZ_GENE_ID': [346389.0, 1.0, 1.0, 29974.0, 29974.0], 'GI': [47271497.0, 21071029.0, 21071029.0, 20357574.0, 20357571.0], 'ACCESSION': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_014576.2'], 'SYMBOL': ['7A5', 'A1BG', 'A1BG', 'A1CF', 'A1CF'], 'PROTEIN_PRODUCT': ['NP_87

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [31]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['ILMN_1762337', 'ILMN_2055271', 'ILMN_1736007', 'ILMN_2383229', 'ILMN_1806310'], 'SPECIES': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'SOURCE': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'SEARCH_KEY': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_138933.1'], 'TRANSCRIPT': ['ILMN_183371', 'ILMN_175569', 'ILMN_18893', 'ILMN_18532', 'ILMN_7300'], 'ILMN_GENE': ['7A5', 'A1BG', 'A1BG', 'A1CF', 'A1CF'], 'SOURCE_REFERENCE_ID': ['NM_182762.2', 'NM_130786.2', 'NM_130786.2', 'NM_138932.1', 'NM_014576.2'], 'REFSEQ_ID'

In [32]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [33]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [34]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [35]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 71 samples.


In [36]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Susceptibility-to-Infections', the least common label is '1.0' with 25 occurrences. This represents 35.21% of the dataset.
The distribution of the feature 'Susceptibility-to-Infections' in this dataset is fine.

Quartiles for 'Age':
  25%: 72.5
  50% (Median): 78.0
  75%: 83.0
Min: 62.0
Max: 107.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 35 occurrences. This represents 49.30% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [37]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: /Users/wjt/Desktop/is_data/v1/Lisa/Susceptibility-to-Infections/cohort_info.json


In [38]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [39]:
from utils import *

In [40]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE150133'

In [41]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE150133,True,True,False,True,True,71,


In [42]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [43]:
merged_data.head()

,Susceptibility-to-Infections,Age,Gender,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,RAB1C
0,1.0,79.0,1.0,2.55465,-1.513733,-0.2121,-0.7710,-4.65425,-0.1748,5.7978,...,-0.470675,3.43186,6.2049,24.6588,-3.02405,186.4947,522.94515,123.5727,91.54825,-0.9349
1,1.0,76.0,0.0,6.97155,-2.274900,-3.8881,-7.2342,-0.52080,-2.1910,8.9607,...,0.694500,1.31132,10.7959,15.5497,-0.01155,198.0979,504.74745,71.2454,120.31650,-4.3098
2,0.0,77.0,1.0,8.01285,-0.440200,-0.2661,-0.0674,1.36105,1.2524,3.5931,...,3.183275,3.04162,9.1069,25.3876,3.53585,218.4672,507.67275,156.8988,94.77770,-1.7484
3,0.0,75.0,1.0,1.31755,-0.778400,-4.0751,1.1207,-0.91295,0.1259,2.8871,...,1.390325,2.70684,10.1412,20.3440,-4.12215,120.1690,448.59990,105.7846,130.35575,-6.4102
4,1.0,81.0,1.0,5.10890,0.347167,-1.7918,-6.7505,-2.18955,-4.6765,11.9485,...,-0.184350,-1.29648,21.2177,20.5295,0.78075,303.7070,599.65585,134.6632,125.67620,-5.2898


In [44]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [45]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [46]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [47]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [48]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 70.00% ± 7.00%


In [49]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 50.00% ± 9.04%


In [50]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [51]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0042
Found 60 genes with non-zero coefficients associated with the trait 'Susceptibility-to-Infections' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
     RBAK    12.987632             12.987632
      C1S    -8.462911              8.462911
   CCDC43     8.325433              8.325433
  RTN4IP1     7.367730              7.367730
  SNORA56     6.476671              6.476671
    IKBIP    -5.378439              5.378439
     TCHH     4.844724              4.844724
     NPFF     4.692095              4.692095
   CAPN10    -4.382461              4.382461
   ABHD15     4.071260              4.071260
  PRAMEF9     3.994500              3.994500
     DCPS     3.826339              3.826339
   TSPYL2     3.758257              3.758257
     MC4R    -3.549325              3.549325
    GNAI3    -3.267754              3

/Users/wjt/Desktop/TAIS_2024SP/Lisa/utils.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [52]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0042
Found 0 significant genes associated with the trait 'Susceptibility-to-Infections' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/Users/wjt/Desktop/TAIS_2024SP/Lisa/utils.py:495: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
